# Load and initialize

settings

In [ ]:
# sample number
j=str(5)
n_samples = 20000


# plots:
    # plot dir
dir_plots = 'Plots_sample'+j+'/'
dir_data = 'Data/'

    # create visualization
visualize = True
    # show figures
show = True

# subsample = False



# umap_iter = False
# dbscan_iter = False
# no_iter = False

envirement

In [ ]:


#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 22 20:09:58 2021

@author: ronguy
"""
import time 
start = time.process_time()

import sys
from IPython.display import Image, display
from tqdm import tqdm_notebook,tqdm
# import time

import numpy as np
# import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
# import matplotlib.patches as  mpatches
plt.rcParams["figure.figsize"] = (5.0, 4.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 180) #according to screen width
# from pandas.core.base import PandasObject
# PandasObject.view = view #allows view meth pd.view

# from scipy import signal, stats
# import seaborn as sns

# from sklearn import metrics
from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans
# from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
# from sklearn.metrics import silhouette_samples,silhouette_score
# from sklearn.neighbors import kneighbors_graph

# from lmfit import minimize, Parameters
from umap import UMAP

import scanpy as sc
import anndata

# import schist as scs

# import networkx as nx

# from castle.common import GraphDAG
# from castle.metrics import MetricsDAG
# from castle.datasets import IIDSimulation, DAG
# from castle.algorithms import PC,Notears,GOLEM,ANMNonlinear,DirectLiNGAM,ICALiNGAM,NotearsLowRank
# import notears.notears as notears

# from shapely.geometry import Point
# from shapely.geometry.polygon import Polygon

# import keras
# from keras import layers
# from keras.callbacks import EarlyStopping
# from keras.callbacks import ModelCheckpoint

# from statsmodels.graphics._regressionplots_doc import _plot_influence_doc
# from statsmodels.regression.linear_model import OLS
# from statsmodels.stats.outliers_influence import variance_inflation_factor



from usefull_functions import *
from functions import *
from normalization import *


%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

folderExists(dir_plots)



load data; normalized (CD45-)  

In [ ]:
dict = pickle_load('k'+j+'_dict',dir_data )
k_orig = dict['k']

# downsample
k = k_orig
k_orig= subsample_k(k.copy(),n=n_samples) 
del k



NamesAll = dict['names']['NamesAll']
ToNorm = dict['names']['ToNorm']
CellIden = dict['names']['CellIden']
EpiCols = dict['names']['EpiCols']
NMS = dict['names']['NMS']
all = dict['all']

del dict

Visualize data

no point to print data.describe() as it is whitened (std~1, mean~0) 

In [ ]:
if visualize:
  for name in [NamesAll,ToNorm,CellIden,EpiCols,NMS]:
    
    print(name)
    print('\n')
  # print(k.min().T)

# Cell Identity

In [ ]:
k = k_orig.copy()
CAll=pd.concat([k]).copy()
s ={}
s['min_dist'] = {}
s['n_neighbors'] = {}
s['min_samples'] = {}
s['eps'] = {}
s['xsize'] = {}


### UMAP

plot for each feature according to its intensity

In [ ]:
s['min_dist']['1']=0.03
s['n_neighbors']['1']=100
# 0.05,150
s['min_dist']['2']=0.001
s['n_neighbors']['2']=60

s['min_dist']['4']=0.001
s['n_neighbors']['4']=60

s['min_dist']['5']=0.001
s['n_neighbors']['5']=60

min_dist = s['min_dist'][j]
n_neighbors = s['n_neighbors'][j]



X_2d = calculate_umap(CAll[CellIden],n_neighbors, min_dist)
if visualize:
  draw_umap (X_2d,CAll['H4'],dir_plots,show = True,
            title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
            figname = 'Tumor'+j+'_UMAP_CellIdentity_H4(core)')



# umap_params(k,CellIden,dir_plots,
#             valA = 0.05, valB = 150,
#             )




In [ ]:
if visualize:
    plot_x2d(X_2d, NamesAll,CAll,dir_plots,show,
            title ='Tumor'+j+ ' Cell Iden UMAP', 
            figname = 'Tumor'+j+'_UMAP_CellIdentity_')

### Dbscan and clusters

In [ ]:
s['min_samples']['1']=80
s['eps']['1']=0.35

s['min_samples']['2']=100
s['eps']['2']=0.2

s['min_samples']['4']=100
s['eps']['4']=0.2

s['min_samples']['4']=100
s['eps']['4']=0.2

s['min_samples']['5']=100
s['eps']['5']=0.2

min_samples = s['min_samples'][j]
eps = s['eps'][j]





X,labels,core_samples_mask = calculate_dbscan(data = X_2d,eps=eps,min_samples=min_samples)
if visualize:
  plot_dbscan(X,labels,core_samples_mask,dir_plots,show,
              title=f'min_samples = {min_samples}, eps = {eps}',
              figname='Tumor'+j+'_dbscan_CellIdentity'
              )
  


# dbscan_params(X_2d,
#                   valA = eps, valB = min_samples,
#                   itersA = 3, itersB = 3,
#                   )


In [ ]:

plotClusters(k,X_2d,labels,NamesAll,dir_plots,
             title = 'Clusters Tumor '+j+' - Cell Iden Based',
             figname = 'Tumor' +j+'_Clusters_CellIden' )



### corr mats

In [ ]:
if visualize:
  corrMat(k,CellIden,dir_plots,
          title = 'T'+j+' Cell Iden Based',
          figname = 'Tumor' +j+'_corrMat_CellIden' )
  corrMat(k,NMS,dir_plots,
          title = 'T'+j+' Cell Iden Based',
          figname = 'Tumor' +j+'_CorrMat_CellIden(NMS)' )




### MeanDist

In [ ]:
s['xsize']['1'] = 20
s['xsize']['2'] = 15
s['xsize']['4'] = 15
s['xsize']['4'] = 15
if visualize:
        MeanDist(k [k.Clust==0],k [k.Clust==1],CellIden,dir_plots,
                title='MeanDist CellIdentity',
                figname = 'Tumor'+j+'_MeanDist_CellIdentity',
                xsize = s['xsize'][j])





In [ ]:
try: 
  del X, X_2d,k, CAll,labels,s
except:
  pass


# Epigenetics Based

In [ ]:

MRK=EpiCols.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')

k = k_orig.copy()
CAll=pd.concat([k]).copy()

s ={}
s['min_dist'] = {}
s['n_neighbors'] = {}
s['min_samples'] = {}
s['eps'] = {}
s['xsize'] = {}



### UMAP

plot for each feature according to its intensity

In [ ]:
s['min_dist']['1']=0.05
s['n_neighbors']['1']=30

s['min_dist']['2']=0.05
s['n_neighbors']['2']=30

s['min_dist']['4']=0.001
s['n_neighbors']['4']=10

s['min_dist']['5']=0.001
s['n_neighbors']['5']=10

min_dist = s['min_dist'][j]
n_neighbors = s['n_neighbors'][j]


X_2d = calculate_umap(CAll[MRK],n_neighbors, min_dist)
if visualize:
  draw_umap (X_2d,CAll['H4'],dir_plots,show = True,
            title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
            figname = 'Tumor'+j+'_UMAP_Epigen_H4(core)')


# umap_params(k,MRK,dir_plots,
#             valA = min_dist, valB = n_neighbors,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.2, valB = 300,
#             )
# umap_params(k,MRK,dir_plots,
#             valA = 0.1, valB = 300,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.001, valB = 300,
#             )

In [ ]:

if visualize:
    plot_x2d(X_2d, NamesAll,CAll,dir_plots,show,
         title ='Tumor '+j+' Epigen UMAP', 
         figname = 'tumor'+j+'_UMAP_Epigen_')
    
    


### Dbscan and clusters

In [ ]:
   
s['min_samples']['1']=60
s['eps']['1']=0.35

s['min_samples']['2']=100
s['eps']['2']=0.2

s['min_samples']['4']=200
s['eps']['4']=0.3

s['min_samples']['5']=200
s['eps']['5']=0.3

s['min_samples']['4']=100
s['eps']['4']=0.2

min_samples = s['min_samples'][j]
eps = s['eps'][j]


X,labels,core_samples_mask = calculate_dbscan(data = X_2d,eps=eps,min_samples=min_samples)
if visualize:
  plot_dbscan(X,labels,core_samples_mask,dir_plots,show,
              title=f'min_samples = {min_samples}, eps = {eps}',
              figname='Tumor'+j+'_dbscan_Epigen'
              )
        
        

# dbscan_params(X_2d,dir_plots,
#                   valA = eps, valB = min_samples,
#                   itersA = 3, itersB = 3,
#                   )


  
 

In [ ]:
plotClusters(k,X_2d,labels,NamesAll,dir_plots,
             title = 'Clusters Tumor '+j+' - Epigen Based',
             figname = 'Tumor' +j+'_Clusters_Epigen' )


## corr mats

In [ ]:
if visualize:
  corrMat(k,EpiCols,dir_plots,
          title = 'T'+j+' Epigen Based',
          figname = 'Tumor' +j+'_CorrMat_Epigen' )


## Meandist

In [ ]:
s['xsize']['1'] = 10
s['xsize']['2'] = 10
s['xsize']['4'] = 10
s['xsize']['5'] = 10
if visualize:
        MeanDist(k [k.Clust==0],k [k.Clust==1],CellIden,dir_plots,
                title='MeanDist CellIdentity',
                figname = 'Tumor'+j+'_MeanDist_CellIdentity',
                xsize = s['xsize'][j])


In [ ]:
try: 
  del X, X_2d,k, CAll,labels,s
except:
  pass